为了使用Omnisafe对论文“Is Conditional Generative Modeling all you need for Decision-Making?”中提出的Decision Diffusion（DD)方法进行复现，首先需要了解Omnisafe框架的运行模式，然后在根据论文中对DD method的描述以及作者团队给出的代码，移植到omnisafe库中。

# 1 omnisafe框架梳理

通过阅读Omnisafe框架文档与自行运行omnisafe提供的example程序进行debug拆解、分析，我总结了Omnisafe框架的架构设计和运行逻辑。
接下来，我将以 `omnisafe\examples\train_from_yaml.py`为例结合源代码和文字说明来梳理一个算法是如何在omnisafe框架下实现的。**需要注意的是，源代码引用部分我将仅保留我认为与新算法复现有关的部分。
为了便于理解，我以BCQ算法为例绘制了一张图来描述omnisafe各模块之间的关系

![omnisafeBCQ](./img/omnisafe.png "omnisafeBCQ")

In [ ]:
import omnisafe

if __name__ == '__main__':
    env_id = 'SafetyAntVelocity-v1'
    algo = "BCQ"
    agent = omnisafe.Agent(algo, env_id)
    agent.learn()
    agent.plot(smooth=1)
    agent.render(num_episodes=1, render_mode='rgb_array', width=256, height=256)
    agent.evaluate(num_episodes=1)

首先定义了`env_id`和`algo`这两个字符串，它们分别代表环境和算法的名称，对于`algo`字符串的内容，要求可以在`omnisafe.algorithms`下能寻找到对应名称的`.py`文件。
随后，这两个字符串作为一个类`class AlgoWrapper`的实例化参数传输给了`omnisafe.Agent(algo,env_id)`,因此，变量`agent`是`class AlgoWrapper`的实例化对象。
接下来我们关注`class AlgoWrapper`的实例化过程(即`__init__`)。

In [ ]:
class AlgoWrapper:
    def _init_config(self) -> Config:
    
    def _init_algo(self) -> None:


class BaseAlgo(ABC):


class BaseOffline(BaseAlgo):
    def _init(self):
        
    def _init_env(self):

    def _init_logger(self):

    def learn(self):


class BCQ(BaseOffline):
    def _train(self):
        
    def _init_model(self):


函数`def _init_config`返回了一个与给定算法相关config信息。这些config信息来自于`omnisafe.configs`下`{算法名}.yaml`文件，这些文件以yaml格式定义了不同算法需要的配置信息。
函数`def _init_algo`使用算法的config信息和环境名实例化了一个`Class BaseAlgo`并赋值给`self.agent`。
类`class BaseAlgo`涉及到了omnisafe对于RL算法实现的设计理念。omnisafe对于不同RL算法的实现是**层层递进**的，众所周知，RL算法可以细分为几个大类，如off-policy,on-policy,offline,online,model-base,model-free，尽管分成了不同的类型，由于它们都是RL算法，因此这些算法之间仍然存在某些共性，omnisafe将这些共性抽象出来设计了一个”基础的算法类“`BaseAlgo`，然后omnisafe对不同类型的算法设计了继承`BaseAlgo`的子类，e.g. `class Baseoffline(BaseAlgo)`。具体到某些算法，如BCQ，CRR，它们同属offline算法，omnisafe就可以针对这些具体的算法设计出对应的类`class BCQ(Baseoffline)`。这种层层递进式的算法实现方法既符合学习者对RL算法的认知，又符合程序语言中”继承“这一要素的设计逻辑，使得omnisafe十分容易理解和使用。对omnisafe整个框架的熟悉过程中，我发现这种层层递进式的派生思想体现在整个框架的方方面面，如TD3算法是基于DDPG的改良，omnisafe在实现TD3时就直接继承了DDPG，很多Module模块的设计也是由基础模块一步一步派生出来（actor->gaussian_actor）等等。

# 2 Decision Diffuser(DD)算法结构及代码梳理
这部分内容我将结合论文和作者团队给出的代码对DD算法进行详细的梳理，力求弄清楚算法的每一个步骤和细节。
*（tips:此部分内容我将着重于对算法流程和代码进行描述，因此我假定学习者已经阅读过论文”IS CONDITIONAL GENERATIVE MODELING ALL YOU NEED FOR DECISION-MAKING?“并对该算法已有基础的了解）*

![algo](./img/algo.png "DDalgo")

DD算法可以分为三部分：数据集构造，前向传播（决策过程）和损失函数构造（反向传播）。
***tips:注意！DD算法的前向传播和反向传播需要与DiffusionModel的前向扩散（加噪）和逆向扩散（去噪）相区别。前向传播时使用的时DiffusionModel的逆向去噪，构造损失函数时使用的是加噪。***
## 2.1 数据集构造
类似于其他OfflineRL算法，DD算法需要构造一个含有大量trajectories的数据集。
> 根据作者团队给出的代码，数据集中一条数据的格式为$[Trajectory,Returns,Condition]$
> $Trajectory$是一个$size为Horizon*(StateDim+ActionDim)$的matrix，矩阵的每一行表示一个state与action的拼接向量（代码中为了方便在实验过程验证DD的其他变体使用的是$[state,aciton]$,但是实际上DD算法模型在forward的过程中只输入了$[state]$），列数Horizon（这是一个超参数）表示轨迹的长度，
> $Returns$是一个数值（1*1的matrix），为该条数据中$Trajectory$对应的累计回报，由$Trajectory$每个state对应的奖励值reward和一个超参数$\gamma$（即强化学习中的折扣因子）计算得到。
> $Condition$是一个字典{int:Tensor},字典的key表示条件的序号，value表示该条件的值，文章中使用one-hot编码来表示条件向量，条件向量可以用来表示和$Trajectory$相关的某些约束。
## 2.2 前向传播/决策过程
作为一个RL算法，DD前向传播的目的是根据当前对环境的观测值$obs$输出动作值$action$。
算法的4-12行描述了这一过程。
>4:此处从环境进行观测获取状态s，并将s插入到一个队列h中，h会保存最近的C个历史状态。（ps:在作者提供的代码中，C=1，即h只会保存当前观测的状态s。）然后从一个标准正态分布中采样出$x_T$,即diffsion过程中的最后一个状态。

In [ ]:
    '''
    这一行对应了以下代码，路径diffuser.model.diffsion.GaussianInvDynDiffusion.p_sample_loop()
    '''''
def p_sample_loop(self, shape, cond, returns=None, verbose=True, return_diffusion=False):
    device = self.betas.device
    batch_size = shape[0]
    x = 0.5 * torch.randn(shape, device=device)
    x = apply_conditioning(x, cond, 0)  #此处实现的是伪代码第六行

>5,10:进入diffsion去噪循环，K为超参数，指定了去噪总步数。
>
>6:将h赋值给$x_k$的前C个状态
>
>7:使用文章中提到的TemporalUnet模型采样出噪声

In [ ]:
    '''此处对应伪代码中的第7行，路径diffuser.model.diffsion.GaussianInvDynDiffusion.p_mean_variance()'''


def p_mean_variance(self, x, cond, t, returns=None):
    if self.returns_condition:
        # epsilon could be epsilon or x0 itself
        epsilon_cond = self.model(x, cond, t, returns, use_dropout=False)  #这一项对应着使用约束条件的epsilon
        epsilon_uncond = self.model(x, cond, t, returns, force_dropout=True)  #这一项对应着不使用约束条件的epsilon（约束条件被设置为空集）
        epsilon = epsilon_uncond + self.condition_guidance_w * (epsilon_cond - epsilon_uncond)  #


'''该forward函数为即上述代码的self.model(),路径diffuser.model.temporal.TemporalUnet.forward()'''


def forward(self, x, cond, time, returns=None, use_dropout=True, force_dropout=False):
    '''
        x : [ batch x horizon x transition ]
        returns : [batch x horizon]
    '''
    if self.calc_energy:
        x_inp = x

    x = x.permute(0, 2, 1)
    # x = einops.rearrange(x, 'b h t -> b t h')

    t = self.time_mlp(time)

    if self.returns_condition:
        assert returns is not None
        returns_embed = self.returns_mlp(returns)
        if use_dropout:
            mask = self.mask_dist.sample(sample_shape=(returns_embed.size(0), 1)).to(returns_embed.device)
            returns_embed = mask * returns_embed
        if force_dropout:
            returns_embed = 0 * returns_embed  #可以看到，此处returns相关的信息被遮蔽了


>8-9:利用采样出的噪声去噪后得到下一步分布的均值和方差并从中采样出($x_{k-1}$)。这部分内容涉及到了论文中省略的diffsionModel的去噪采样过程，在此补充上相关数学推导。
![math2](./img/diffmath2.png "diffmath2")
![math3](./img/diffmath3.png "diffmath3")
![math1](./img/diffmath1.png "diffmath1")

In [ ]:
'''
以下代码对应了上述数学推导，为了与推导过程一一对应，我选取了不同部分拼接
'''

'''
diffuser.model.diffsion.GaussianInvDynDiffusion.p_mean_variance
'''
x_recon = self.predict_start_from_noise(x, t=t, noise=epsilon)  #计算的到x0

'''
diffuser.model.diffsion.GaussianInvDynDiffusion.predict_start_from_noise
'''


def predict_start_from_noise(self, x_t, t, noise):
    '''
        if self.predict_epsilon, model output is (scaled) noise;
        otherwise, model predicts x0 directly
    '''
    if self.predict_epsilon:
        return (
            extract(self.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t -
            extract(self.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise
        )
    else:
        return noise


'''
diffuser.model.diffsion.GaussianInvDynDiffusion.p_mean_variance
'''
model_mean, posterior_variance, posterior_log_variance = self.q_posterior(
    x_start=x_recon, x_t=x, t=t)

'''
diffuser.model.diffsion.GaussianInvDynDiffusion.q_posterior
'''


def q_posterior(self, x_start, x_t, t):
    posterior_mean = (
        extract(self.posterior_mean_coef1, t, x_t.shape) * x_start +
        extract(self.posterior_mean_coef2, t, x_t.shape) * x_t
    )  #计算得到均值ut，对应于公式(8-2)
    posterior_variance = extract(self.posterior_variance, t, x_t.shape)
    posterior_log_variance_clipped = extract(self.posterior_log_variance_clipped, t, x_t.shape)  #计算得到方差sigma_t
    return posterior_mean, posterior_variance, posterior_log_variance_clipped


'''
diffuser.model.diffsion.GaussianInvDynDiffusion.p_sample
'''
model_mean, _, model_log_variance = self.p_mean_variance(x=x, cond=cond, t=t, returns=returns)
noise = 0.5 * torch.randn_like(x)
model_mean + nonzero_mask * (0.5 * model_log_variance).exp() * noise  #采样得到X_{t-1}

>11-12:通过参数化的逆动力学模型使用$s_t$和$s_{t+1}$推断出$a_t$

In [ ]:
'''
路径：diffuser.model.diffsion.GaussianInvDynDiffusion.predict
'''
obs_comb = torch.cat([samples[:, 0, :], samples[:, 1, :]], dim=-1)  #获取s_t与s_{t+1}
obs_comb = obs_comb.reshape(-1, 2 * self.observation_dim)
action = self.inv_model(obs_comb)  #通过逆动力学模型推断action_t

至此，DD算法的前向传播（即forward或决策过程）梳理完毕。

## 2.3 损失函数构造/反向传播
![loss](./img/loss.png "loss")
损失函数分为两部分，一部分是diffsion过程产生的loss值$Loss_{diff}$，另一部分是求解逆动力学模型产生的loss值$Loss_{inv}$。
在计算$Loss_{diff}时，作者使用了一个trick：以一定概率丢失掉条件信息condition。

In [ ]:
'''
路径：diffuser.model.diffsion.GaussianInvDynDiffusion.p_loss
功能：计算来自diffsion的loss值
'''
def p_losses(self, x_start, cond, t, returns=None):
    noise = torch.randn_like(x_start)  #此处采样一个标准高斯分布的噪声

    x_noisy = self.q_sample(x_start=x_start, t=t, noise=noise)
    x_noisy = apply_conditioning(x_noisy, cond, 0)

    x_recon = self.model(x_noisy, cond, t, returns)  #此处通过参数化的采样模型TemporlUnet得到epsilon_theta

    if not self.predict_epsilon:
        x_recon = apply_conditioning(x_recon, cond, 0)

    assert noise.shape == x_recon.shape

    if self.predict_epsilon:
        loss, info = self.loss_fn(x_recon, noise)  #计算Loss_{diff}的值
    else:
        loss, info = self.loss_fn(x_recon, x_start)

    return loss, info


'''
该forward函数为即上述代码的self.model()
路径diffuser.model.temporal.TemporalUnet.forward()
'''
def forward(self, x, cond, time, returns=None, use_dropout=True, force_dropout=False):
    '''
        x : [ batch x horizon x transition ]
        returns : [batch x horizon]
    '''
    if self.calc_energy:
        x_inp = x

    x = x.permute(0, 2, 1)
    # x = einops.rearrange(x, 'b h t -> b t h')

    t = self.time_mlp(time)

    if self.returns_condition:
        assert returns is not None
        returns_embed = self.returns_mlp(returns)
        if use_dropout:  #此处即以一定概率丢弃condition条件信息，这里的条件信息时returns
            mask = self.mask_dist.sample(sample_shape=(returns_embed.size(0), 1)).to(
                returns_embed.device)  #生成mask进行condition遮蔽
            returns_embed = mask * returns_embed


'''
路径：diffuser.model.diffsion.GaussianInvDynDiffusion.loss
功能：计算总loss值
'''
def loss(self, x, cond, returns=None):
    batch_size = len(x)
    t = torch.randint(0, self.n_timesteps, (batch_size,), device=x.device).long()
    diffuse_loss, info = self.p_losses(x[:, :, self.action_dim:], cond, t, returns) #计算diffsion的loss值
    # Calculating inv loss
    x_t = x[:, :-1, self.action_dim:]
    a_t = x[:, :-1, :self.action_dim]
    x_t_1 = x[:, 1:, self.action_dim:]
    x_comb_t = torch.cat([x_t, x_t_1], dim=-1)
    x_comb_t = x_comb_t.reshape(-1, 2 * self.observation_dim)
    a_t = a_t.reshape(-1, self.action_dim)
    if self.ar_inv:
        inv_loss = self.inv_model.calc_loss(x_comb_t, a_t)
    else:
        pred_a_t = self.inv_model(x_comb_t)
        inv_loss = F.mse_loss(pred_a_t, a_t)  #此处计算了参数化的逆动力学模型的loss值
    loss = (1 / 2) * (diffuse_loss + inv_loss) #总loss值
    info['loss_diffuser'] = diffuse_loss
    info['loss_inv'] = inv_loss
    info['loss_total'] = loss
    return loss, info

# 3 在Omnisafe框架下对DecisionDiffuser算法的移植
通过对omnisafe框架和DD算法的梳理，移植过程已经呼之欲出。
首先根据omnisafe层层递进派生式的设计思想，需要确定DD算法在RL中所属的门类。
通过对DD算法的理解，我认为其勉强可以算作一种offline算法。至此，为了实现DD算法的移植，我将新建这些代码文件：
新增文件:
   > - DD.py
      > - 路径:omnisafe/algorithms/offline/DD.py
      > - 作用:以omnisafe style实现DD算法的主体。
      > - tips:DD.py的实现参考BCQ.py，我首先完全复制了BCQ.py，然后根据DD算法的代码逐步更改，我认为这是最快的实现方式。

   > - DD.yaml
      > - 路径:omnisafe/config/offline/DD.yaml
      > - 作用:存放DD算法的配置信息。
      > - tips:参考DD算法代码的locomotion_config.py并以omnisafe style实现。
   
   > - dd_models
      > - 路径:omnisafe/models/dd_models
      > - 作用:存放实现DD算法中各个模块的nn.Module类
      > - tips:这部分参考DD算法的源代码。

修改文件:
   > - dataset.py
      > - 新定义了一个类`class DeciDiffuserDataset(offlineDataset)`，该类继承offlineRL算法的数据集操作类`offlineDataset`
      > - 为了与DD原代码中给出的Module所需要的输入参数相匹配，重写了`sample`函数和部分属性。